In [1]:
import os
os.getcwd()
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
from collections import namedtuple

In [3]:
DataIngestionConfig = namedtuple("dataIngestioConfig",
["root_dir","source_URL","local_data_file","unzip_dir"])

In [4]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [5]:
class ConfigrationManger:
    def __init__(self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config



In [8]:
#DataIngestionComponent
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url=self.config.source_URL,
                filename= self.config.local_data_file
            )

    #hidden method to check and take only jpg imgs
    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self,zf:ZipFile,f:str , working_dir:str):
        target_file_path = os.path.join(working_dir,f)
        if  not os.path.exists(target_file_path):
            zf.extract(f,working_dir)

        if os.path.getsize(target_file_path) == 0:
            os.remove(target_file_path)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file,mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files=list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)


In [9]:
try:
    config = ConfigrationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-12-16 01:20:04,642: INFO: common]: yaml file: Configrationfile\config.yaml loaded successfully
[2022-12-16 01:20:04,645: INFO: common]: yaml file: params.yaml loaded successfully
[2022-12-16 01:20:04,649: INFO: common]: created directory at: artifacts
[2022-12-16 01:20:04,651: INFO: common]: created directory at: artifacts/data_ingestion
